In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import yfinance as yf
import pandas as pd
import math
from scipy.stats import norm
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Dropout, Conv1D, MaxPooling1D, Flatten, Attention
from tensorflow.keras.models import Model
from ta.volume import MFIIndicator
from ta.utils import dropna
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [12]:
# Kaggle function

def make_model():
    inp = Input(shape=(128, 10))
    x = Bidirectional(LSTM(128, return_sequences=True))(inp)
    x = Bidirectional(LSTM(32, return_sequences=True))(x)
    x = Attention(128)(x)
    # A intermediate full connected (Dense) can help to deal with nonlinears outputs
    x = Dense(64, activation="relu")(x)
    x = Dense(9, activation="softmax")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [24]:
# My function
def make_model():
    # 1. Input layer for stock data (sequence_length, features)
    inp = Input(shape=(128, 9))  # Example: 128 timesteps, 10 features (adjust as needed)

    # 2. Bidirectional LSTM layers for sequence learning
    x = Bidirectional(LSTM(128, return_sequences=True))(inp)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)

    # 3. Dropout to prevent overfitting
    x = Dropout(0.2)(x)

    # 4. 1D Convolutional layer for feature extraction
    x = Conv1D(filters=64, kernel_size=3, activation="relu", padding="same")(x)

    # 5. MaxPooling to downsample
    x = MaxPooling1D(pool_size=2)(x)

    # 6. Flatten layer to transition to fully connected layers
    x = Flatten()(x)

    # 7. Fully connected layers for high-level learning
    x = Dense(128, activation="relu")(x)
    x = Dense(64, activation="relu")(x)

    # 8. Output layer - Single neuron with tanh activation to produce values between -1 and 1
    output = Dense(1, activation="tanh")(x)

    # 9. Compile model with Adam optimizer
    model = Model(inputs=inp, outputs=output)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

    return model

# Create the model
model = make_model()
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 128, 9)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 128, 256)       │       141,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, 128, 128)       │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 128, 64)        │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 64, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 863,041 (3.29 MB)

 Trainable params: 863,041 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
def calculate_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)
    
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    rsi = rsi.fillna(50)  # Default to neutral if not enough data
    return rsi

In [10]:
def get_stock_data(ticker, start='2024-01-01', end='2025-03-24', buffer_days=200):
    # Extend the start date backward by the buffer_days
    extended_start = pd.to_datetime(start) - pd.Timedelta(days=buffer_days)
    stock = yf.download(ticker, start=extended_start.strftime('%Y-%m-%d'), end=end)
    return stock

In [13]:


def data_preparation(data, start_date='2024-01-01', buffer_days_ma50=50, buffer_days_ma200=200):
    data_ma50 = data[data.index >= (pd.to_datetime(start_date) - pd.Timedelta(days=buffer_days_ma50))]
    data_ma200 = data[data.index >= (pd.to_datetime(start_date) - pd.Timedelta(days=buffer_days_ma200))]
 
    
    data = dropna(data)  # Drop NA values from the dataframe

    data[['Log Price']] = np.log(data['Close'])  # Log transformation for stationarity
    data[['Log Volume']] = np.log(data['Volume'].replace(0,np.nan))  # Log transformation for volume, add 1 to avoid log(0)
    data[['Log Volume']].fillna(data[['Log Volume']].rolling(window=5, min_periods=1).mean(), inplace=True)

    data[['Log Price Diff']] = data[['Log Price']].diff()

    data['Percent Change'] = data['Close'].pct_change()  # Calculate percentage change for the close price

    data['RSI'] = calculate_rsi(data)  # Calculate RSI
    
    mfi = MFIIndicator(
        high=data['High'].squeeze(),
        low=data['Low'].squeeze(),
        close=data['Close'].squeeze(),
        volume=data['Volume'].squeeze(),
        window=14,
        fillna=True)
    data['MFI'] = mfi.money_flow_index()
    
    data['Log Volume Diff'] = data['Log Volume'].diff()  # Log volume difference for stationarity
    
    # Beta calculation (S&P500)
    market = yf.download('SPY', start=data.index[0], end=data.index[-1])['Close']
    data['Market Return'] = market.pct_change()
    data['Stock Return'] = data['Close'].pct_change()
    data['Beta'] = data['Stock Return'].rolling(30).cov(data['Market Return']) / data['Market Return'].rolling(30).var()
    
    data['PE Ratio'] = data['Close'] / data['Earnings'] if 'Earnings' in data.columns else np.nan
     # Example PE ratio calculation, ensure 'Earnings' column exists in your data

    data_ma50['MA50'] = data_ma50['Close'].rolling(50).mean()
    data_ma50['Log Diff MA50'] = np.log(data_ma50['Close'].values.flatten()) - np.log(data_ma50['MA50'].values.flatten())

    data_ma200['MA200'] = data_ma200['Close'].rolling(200).mean()
    data_ma200['Log Diff MA200'] = np.log(data_ma200['Close'].values.flatten()) - np.log(data_ma200['MA200'].values.flatten())



    decomposition = seasonal_decompose(data['Close'], model='additive', period=math.floor(data['Close'].size/2))  # Assuming yearly seasonality
    data['Seasonality'] = decomposition.seasonal

    data = data[data.index >= start_date]

    data = data.merge(data_ma50[['MA50', 'Log Diff MA50']], how='left', left_index=True, right_index=True)
    data = data.merge(data_ma200[['MA200', 'Log Diff MA200']], how='left', left_index=True, right_index=True)

    feature_columns = ['Log Price Diff',  'RSI', 'MFI', 'Log Volume Diff',
                       'Beta', 'PE Ratio', 'Log Diff MA50', 'Log Diff MA200', 'Seasonality']
    
    return data[feature_columns],data['Percent Change']

    # Normalize the data
    #data = (data - data.mean()) / data.std()
    

In [27]:
apple = get_stock_data('AAPL')
apple['Volume']
prepped_data,target = data_preparation(apple)
prepped_data.head()


[*********************100%***********************]  1 of 1 completed
/var/folders/39/953q32fx2pgd35pdr0s2p0r00000gn/T/ipykernel_5506/817621214.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['Log Volume']].fillna(data[['Log Volume']].rolling(window=5, min_periods=1).mean(), inplace=True)
[*********************100%***********************]  1 of 1 completed
/var/folders/39/953q32fx2pgd35pdr0s2p0r00000gn/T/ipykernel_5506/817621214.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ma50['MA50'] = data_ma50['Close'].rolling(50).mean()
/var/folders/39

Price,Log Price Diff,RSI,MFI,Log Volume Diff,Beta,PE Ratio,Log Diff MA50,Log Diff MA200,Seasonality
Ticker,,,,,,,,,
Date,,,,,,,,,
2024-01-02,-0.036443,31.663456,33.908998,0.660131,1.110405,NaN,NaN,NaN,4.537039
2024-01-03,-0.007516,24.387807,27.139145,-0.345097,1.066214,NaN,NaN,NaN,3.093644
2024-01-04,-0.012781,8.867208,18.205990,0.208874,1.072855,NaN,NaN,NaN,0.673518
2024-01-05,-0.004021,7.864678,9.674153,-0.144424,1.075951,NaN,NaN,NaN,-0.139849
2024-01-08,0.023887,24.906009,18.681121,-0.052031,1.214553,NaN,NaN,NaN,4.137002


In [17]:


X_train, X_temp, y_train, y_temp = train_test_split(prepped_data, target, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)  # 20% val, 10% test


In [19]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [20]:
BATCH_SIZE = 32  # Adjust based on dataset size

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE).shuffle(1000)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)


In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  
              loss='mse',   # Mean Squared Error (common for regression)
              metrics=['mae', 'mape'])  # Mean Absolute Error & Mean Absolute Percentage Error


In [26]:
history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val), 
                    epochs=50, 
                    batch_size=32, 
                    verbose=1)

Epoch 1/50


ValueError: Input 0 of layer "functional_2" is incompatible with the layer: expected shape=(None, 128, 9), found shape=(None, 9)